In [1]:
import sys
from pathlib import Path
import os

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/isabella/Documents/KTH/ScalableML/mlfs-book_scalabel_ml
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [3]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-18 23:36:21,110 INFO: Initializing external client
2025-11-18 23:36:21,110 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-18 23:36:22,588 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279144


'{"country": "sweden", "city": "stockholm", "street": "hornsgatan-108", "aqicn_url": "https://api.waqi.info/feed/@10220/", "latitude": "59.314017450051836", "longitude": "18.075630285191593"}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [4]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=2,  # Using version 2 with lagged features
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [5]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,NaN,sweden,stockholm,hornsgatan-108,2025-11-18,https://api.waqi.info/feed/@10220/


In [6]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     0 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🔄 Add Lagged Features </span>

To create lagged features for today's data, we need to fetch the previous 3 days of PM2.5 measurements from the feature store.

In [7]:
# Fetch the last 3 days of PM2.5 data from the feature store to create lagged features
from datetime import timedelta

# Calculate date range: 3 days ago to yesterday
start_date = today - timedelta(days=3)
end_date = today - timedelta(days=1)

# Read historical data from feature store
fg_query = air_quality_fg.select(['date', 'pm25']) \
    .filter(air_quality_fg.country == country) \
    .filter(air_quality_fg.city == city) \
    .filter(air_quality_fg.street == street) \
    .filter(air_quality_fg.date >= start_date) \
    .filter(air_quality_fg.date <= end_date)

try:
    historical_df = fg_query.read()
    historical_df = historical_df.sort_values('date')
    
    if len(historical_df) >= 3:
        # Extract the last 3 days of PM2.5 values
        pm25_values = historical_df['pm25'].tail(3).tolist()
        
        # Assign lagged features to today's data
        aq_today_df['pm25_lag_1d'] = pm25_values[-1] if len(pm25_values) >= 1 else None
        aq_today_df['pm25_lag_2d'] = pm25_values[-2] if len(pm25_values) >= 2 else None
        aq_today_df['pm25_lag_3d'] = pm25_values[-3] if len(pm25_values) >= 3 else None
        
        print(f"✅ Added lagged features from previous 3 days:")
        print(f"   pm25_lag_1d: {aq_today_df['pm25_lag_1d'].values[0]}")
        print(f"   pm25_lag_2d: {aq_today_df['pm25_lag_2d'].values[0]}")
        print(f"   pm25_lag_3d: {aq_today_df['pm25_lag_3d'].values[0]}")
    else:
        print(f"⚠️ Warning: Only {len(historical_df)} days of historical data found. Need at least 3 days.")
        print("   Setting lagged features to None. Please run feature backfill first.")
        aq_today_df['pm25_lag_1d'] = None
        aq_today_df['pm25_lag_2d'] = None
        aq_today_df['pm25_lag_3d'] = None
except Exception as e:
    print(f"⚠️ Could not fetch historical data: {e}")
    print("   Setting lagged features to None.")
    aq_today_df['pm25_lag_1d'] = None
    aq_today_df['pm25_lag_2d'] = None
    aq_today_df['pm25_lag_3d'] = None

aq_today_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.80s) 
⚠️ Warning: Only 0 days of historical data found. Need at least 3 days.
   Setting lagged features to None. Please run feature backfill first.


,pm25,country,city,street,date,url,pm25_lag_1d,pm25_lag_2d,pm25_lag_3d
0,NaN,sweden,stockholm,hornsgatan-108,2025-11-18,https://api.waqi.info/feed/@10220/,None,None,None


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [8]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 59.25°N 18.0°E
Elevation 36.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-18,-1.50,0.0,7.903619,300.068481,stockholm
1,2025-11-19,-1.65,0.2,7.568566,357.273743,stockholm
2,2025-11-20,-0.95,0.1,13.004921,4.763556,stockholm
3,2025-11-21,-1.95,0.0,18.278645,327.875031,stockholm
4,2025-11-22,1.30,0.0,16.548256,225.881302,stockholm
5,2025-11-23,1.30,0.0,9.387651,212.471176,stockholm
6,2025-11-24,1.05,0.0,8.854829,296.564972,stockholm


In [9]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 352.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [10]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-18 23:36:30,016 INFO: 	1 expectation(s) included in expectation_suite.
Validation failed.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279144/fs/1265752/fg/1721799


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279144/jobs/named/air_quality_2_offline_fg_materialization/executions


(Job('air_quality_2_offline_fg_materialization', 'SPARK'),
 {
   "success": false,
   "results": [
     {
       "success": false,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 761950
         }
       },
       "result": {
         "observed_value": null,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T10:36:30.000015Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expect

In [11]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-18 23:37:11,960 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279144/fs/1265752/fg/1596014


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279144/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-18 23:37:29,668 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-18 23:37:32,880 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-18 23:39:17,471 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-18 23:39:17,638 INFO: Waiting for log aggregation to finish.
2025-11-18 23:39:29,674 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 733226
         }
       },
       "result": {
         "observed_value": 7.56856632232666,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T10:37:11.000959Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
